In [160]:
import mysql.connector
import pandas as pd
import numpy as np
import re

In [161]:
mydb = mysql.connector.connect(
    host="localhost",
    user="student",
    password="student")
print(mydb)

In [162]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS proyecto_final")
mycursor.execute("USE proyecto_final")

In [163]:
camposDict = {'arte_diseno_arquitectura':1, 'agricultura_forestal':2, 'empresariales_administracion':3,
             'economicas':4, 'educacion':5, 'ingenieria_tecnologia':6, 'historia_filosofia_teologia':7,
             'leyes_abogacia':8, 'ciencias_salud':9, 'ciencias_naturales': 10, 'ciencias_sociales':11, 'otros':12}

In [164]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Campos (\
                id int PRIMARY KEY, \
                nombre varchar(255))")
sqlCampos = ("INSERT INTO Campos (id, nombre) VALUES (%s, %s)")
val = []
for k, v in camposDict.items():
    val.append((v, k))
mycursor.executemany(sqlCampos, val)
mydb.commit()

In [165]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Universidades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                url varchar(255), \
                posicion_hindex int, \
                tipo_centro varchar(255))")

In [166]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Facultades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                id_univ int, \
                id_campo int, \
                FOREIGN KEY (id_univ) REFERENCES Universidades(id) ON DELETE CASCADE ON UPDATE CASCADE, \
                FOREIGN KEY (id_campo) REFERENCES Campos(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [167]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Grados (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                id_fac int, \
                nombre varchar(255), \
                presencial bool, \
                nota_corte double, \
                idioma bool, \
                nota_corte_ant double, \
                nota_admision double, \
                rendimiento double, \
                creditos double, \
                precio_creditos double, \
                empleabilidad double, \
                FOREIGN KEY (id_fac) REFERENCES Facultades(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [168]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Rankings (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                id_univ int, \
                id_campo int, \
                puntuacion int, \
                FOREIGN KEY (id_univ) REFERENCES Universidades(id) ON DELETE CASCADE ON UPDATE CASCADE, \
                FOREIGN KEY (id_campo) REFERENCES Campos(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [169]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('Campos',)
('Facultades',)
('Grados',)
('Rankings',)
('Universidades',)


In [170]:
df = pd.read_csv("file:/home/student/Downloads/Datos_QEDU.csv").replace(to_replace='\(.+\)', value = '', regex = True)\
.drop_duplicates(subset=['NOMBRE_UNIV', 'NOMBRE_CENTRO', 'NOM_TITULA']).reset_index()
df_rank = pd.read_csv("file:/home/student/Downloads/uni_rankings_final.csv").replace(-1, np.nan)
df_rank.rename(columns={'nombre_regional': 'NOMBRE_UNIV'}, inplace=True)
df['NOMBRE_UNIV'] = df['NOMBRE_UNIV'].apply(str.strip)
df_rank['NOMBRE_UNIV'] = df_rank['NOMBRE_UNIV'].apply(str.strip)

In [171]:
df2 = df.replace(['Pública / Propio', 'Pública / Adscrito**'], 'Pública').replace(['Privada / Propio', 'Privada / Adscrito**'], 'Privada').fillna(np.nan).replace([np.nan], [None])
dfUniv = df2.groupby(['NOMBRE_UNIV', 'URL_INFO', 'TIPO_UNIV_CENTRO'], dropna=False).size().reset_index().iloc[:, :-1]
df_join = df_rank.merge (dfUniv, on= 'NOMBRE_UNIV', how='inner')
dfNone_join= df_join.fillna(np.nan).replace([np.nan], [None])
dfNone_join

,nombre,nombre_ingles,NOMBRE_UNIV,siglas,cuidad,codigo_postal,posicion_hindex,agricultura_forestal,arte_diseno_arquitectura,empresariales_administracion,...,educacion,ingenieria_tecnologia,historia_filosofia_teologia,leyes_abogacia,ciencias_salud,ciencias_naturales,ciencias_sociales,otros,URL_INFO,TIPO_UNIV_CENTRO
0,Universidad Nacional de Educación a Distancia,National University of Distance Education,Universidad Nacional de Educación a Distancia,UNED,Madrid,28015,57.0,0.0,1.0,1.0,...,4.0,23.0,4.0,5.0,0.0,4.0,13.0,71.0,"http://portal.uned.es/portal/page?_pageid=93,1...",Pública
1,Universidad Abierta de Cataluña,Open University of Cataluña,Universidad Oberta de Catalunya,UOC,Barcelona,8035,38.0,0.0,5.0,12.0,...,9.0,17.0,1.0,5.0,4.0,3.0,39.0,135.0,http://estudios.uoc.edu/es/grados,Privada
2,Universidad Politécnica de Madrid,None,Universidad Politécnica de Madrid,UPM,Madrid,28040,33.0,26.0,7.0,4.0,...,3.0,58.0,1.0,0.0,4.0,19.0,6.0,251.0,http://www.upm.es/institucional/Estudiantes/Es...,Pública
3,Universidad de Barcelona,University of Barcelona,Universidad de Barcelona,UB,Barcelona,8007,1.0,15.0,6.0,9.0,...,21.0,73.0,12.0,16.0,97.0,145.0,70.0,674.0,http://www.ub.edu/dyn/cms/continguts_es/estudi...,Pública
4,Universidad de Granada,University of Granada,Universidad de Granada,UGR,Granada,18010,18.0,0.0,3.0,10.0,...,11.0,19.0,12.0,15.0,14.0,27.0,39.0,249.0,http://www.ugr.es/pages/estudios/grado,Pública
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Universidad Intercontinental de la Empresa,University Intercontinental of Bussines,Universidad Intercontinental de la Empresa,UIE,Galicia,None,None,None,None,None,...,None,None,None,None,None,None,None,None,https://uie.edu/grados/,Privada
84,Universidad Fernando Pessoa,University Fernando Pessoa,Universidad Fernando Pessoa-Canarias,UFP-C,Las Palmas,35450,115.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,6.0,http://www.ufpcanarias.es/,Privada
85,Universidad Internacional Villanueva,University of Villanueva,Internacional Villanueva,UV,Madrid,28034,110.0,0.0,0.0,1.0,...,2.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,https://www.villanueva.edu/,Privada
86,Universidad Internacional de la Empresa,International University of Bussiness,Universidad Internacional de la Empresa,UNIE,Madrid,28036,None,None,None,None,...,None,None,None,None,None,None,None,None,https://www.universidadunie.com/programas/,Privada


In [172]:
sqlUniv = "INSERT INTO Universidades (nombre, url, posicion_hindex, tipo_centro) VALUES (%s, %s, %s, %s)"
for index, row in dfNone_join.iterrows():
    val = (row['NOMBRE_UNIV'], row['URL_INFO'], row['posicion_hindex'], row['TIPO_UNIV_CENTRO'])
    mycursor.execute(sqlUniv, val)
mydb.commit()

In [173]:
dfFac = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO'], dropna=False).size().reset_index().iloc[:, :-1]
dfFac

,NOMBRE_UNIV,NOMBRE_CENTRO
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas
1,CUNEF Universidad,Escuela Politécnica Superior
2,CUNEF Universidad,"Facultad de Empresa, Economía y Derecho"
3,ESIC Universidad,Facultad de Ciencias Jurídicas y Sociales
4,Hesperides,Escuela de Grados
...,...,...
1062,Universitat de València,Facultad de Psicología y Logopedia
1063,Universitat de València,Facultad de Química
1064,Universitat de València,Facultat d'Infermeria i Podologia
1065,Universitat de València,Facultat de Fisioteràpia


In [174]:
campos = []
arte_diseno_arquitectura = ['Arquitectura', 'Diseño', 'Arte']
agricultura_forestal = ['Agricultura', 'Forestal']
empresariales_administracion = ['Empresa', 'Empresariales', 'Relaciones', 'Marketing', 'Comercio']
economicas = ['Económicas', 'Finan']
educacion = ['Educación', 'Enseñanza', 'Magisteri', 'Profesorado']
ingenieria_tecnologia = ['Ingeniería', 'Politécnica', 'Informática', 'Náutica', 'Ingeniero', 'Tecnología']
historia_filosofia_teologia = ['Filosofía', 'Historia']
leyes_abogacia = ['Derecho', 'Jurídicas']
ciencias_salud = ['Psicología', 'Salud', 'Medicina', 'Farmacia', 'Biol', 'Óptica', 'Sociosanitarias',
                  'Alimentación', 'Deporte', 'Odontología', 'Enfermería', 'Fisio', 'Infermeria']
ciencias_sociales = ['Comunicación', 'Humanidades', 'Sociología', 'Humana', 'Turismo', 'Criminología', 'Letras',
                     'Social', 'Filología', 'Lengua', 'Traducción', 'Información' 'Documentación', 'Política']
ciencias_naturales = ['Física', 'Matemáticas', 'Veterinaria', 'Naturaleza', 'Ciencias', 'Bioq',
                      'Geolo', 'Química', 'Estadísti']
for index, row in dfFac.iterrows():
    facultad = row['NOMBRE_CENTRO']
    if any([x in facultad for x in arte_diseno_arquitectura]):
        campos.append('arte_diseno_arquitectura')
    elif any([x in facultad for x in agricultura_forestal]):
        campos.append('agricultura_forestal')
    elif any([x in facultad for x in empresariales_administracion]):
        campos.append('empresariales_administracion')
    elif any([x in facultad for x in economicas]):
        campos.append('economicas')
    elif any([x in facultad for x in educacion]):
        campos.append('educacion')
    elif any([x in facultad for x in ingenieria_tecnologia]):
        campos.append('ingenieria_tecnologia')
    elif any([x in facultad for x in historia_filosofia_teologia]):
        campos.append('historia_filosofia_teologia')
    elif any([x in facultad for x in leyes_abogacia]):
        campos.append('leyes_abogacia')
    elif any([x in facultad for x in ciencias_salud]):
        campos.append('ciencias_salud')
    elif any([x in facultad for x in ciencias_naturales]):
        campos.append('ciencias_naturales')
    elif any([x in facultad for x in ciencias_sociales]):
        campos.append('ciencias_sociales')
    else:
        campos.append('otros')
dfFac2 = dfFac.assign(CAMPOS=campos)

In [175]:
sqlFac = "INSERT INTO Facultades (nombre, id_univ, id_campo) VALUES (%s, %s, %s)"
sqlQueryUniv = "SELECT id FROM Universidades WHERE nombre = %s"
uniAnterior = ""
idUni = -1
for index, row in dfFac2.iterrows():
    uniActual = row['NOMBRE_UNIV']
    if uniAnterior != uniActual:
        mycursor.execute(sqlQueryUniv, (uniActual, ))
        myresult = mycursor.fetchall()
        idUni = myresult[0][0]
        uniAnterior = uniActual
    val = (row['NOMBRE_CENTRO'], idUni, camposDict.get(row['CAMPOS']))
    mycursor.execute(sqlFac, val)
mydb.commit()

In [176]:
dfGrad = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO', 'NOM_TITULA', 'PRESENCIALIDAD', 'NOTA_CORTE', 'IDIOMAEXTRANJERO',
                      'NOTA_CORTE_ANT','NOTA_ADMISION', 'RENDIMIENTO',
                      'CREDITOS', 'PRECIO', 'TASA_AFIL'], dropna=False).size().reset_index().iloc[:, :-1]
dfNone= dfGrad.fillna(np.nan).replace([np.nan], [None])
dfNone

,NOMBRE_UNIV,NOMBRE_CENTRO,NOM_TITULA,PRESENCIALIDAD,NOTA_CORTE,IDIOMAEXTRANJERO,NOTA_CORTE_ANT,NOTA_ADMISION,RENDIMIENTO,CREDITOS,PRECIO,TASA_AFIL
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Administración y Dirección de Empresas,Sí,None,No,None,None,87.0,240.0,None,None
1,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Cine,Sí,None,No,None,None,None,240.0,None,None
2,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Comunicación,Sí,None,No,None,None,94.0,240.0,None,None
3,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Derecho,Varias modalidades*,None,No,None,None,23.0,240.0,None,None
4,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Maestro en Educación Infantil,Varias modalidades*,None,No,None,None,96.0,240.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3565,Universitat de València,Facultat d'Infermeria i Podologia,Grado en Enfermería,Sí,12.35,No,12.51,None,90.0,240.0,18.0,78.3
3566,Universitat de València,Facultat d'Infermeria i Podologia,Grado en Podología,Sí,11.03,No,11.08,11.08,87.0,240.0,18.0,80.7
3567,Universitat de València,Facultat de Fisioteràpia,Grado en Fisioterapia,Sí,11.87,No,12.05,None,91.0,240.0,18.0,76.4
3568,Universitat de València,Facultat de Magisteri,Grado en Maestro/a en Educación Infantil,Sí,9.59,No,9.75,10.37,96.0,240.0,12.79,77.1


In [177]:
sqlGrad = "INSERT INTO Grados (id_fac, nombre, presencial, nota_corte, idioma, \
                                    nota_corte_ant, nota_admision, rendimiento, creditos, precio_creditos, \
                                    empleabilidad) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sqlQueryFac = "SELECT id FROM Facultades WHERE id_univ = (SELECT id FROM Universidades WHERE nombre = %s) AND nombre = %s"
uniAnterior = ""
facAnterior = ""
idFac = -1
for index, row in dfNone.iterrows():
    uniActual = row['NOMBRE_UNIV']
    facActual = row['NOMBRE_CENTRO']
    if uniAnterior != uniActual or facAnterior != facActual:
        mycursor.execute(sqlQueryFac, (uniActual, facActual))
        myresult = mycursor.fetchall()
        idFac = myresult[0][0]
        uniAnterior = uniActual
        facAnterior = facActual
    if row['PRESENCIALIDAD']=="No": 
        presencialidad=0 
    else: 
        presencialidad=1
    if row['IDIOMAEXTRANJERO']=="No":
        idioma=0 
    else: 
        idioma=1
    val = (idFac, row['NOM_TITULA'], presencialidad, row['NOTA_CORTE'], 
           idioma, row['NOTA_CORTE_ANT'], row['NOTA_ADMISION'], row['RENDIMIENTO'],
           row['CREDITOS'], row['PRECIO'], row['TASA_AFIL'])
    mycursor.execute(sqlGrad, val)
mydb.commit()

In [178]:
sqlRank = "INSERT INTO Rankings (id_univ, id_campo, puntuacion) VALUES (%s, %s, %s)"
sqlQueryUniv = "SELECT id FROM Universidades WHERE nombre = %s"
for index, row in dfNone_join.iterrows():
    uniActual = row['NOMBRE_UNIV']
    mycursor.execute(sqlQueryUniv, (uniActual, ))
    myresult = mycursor.fetchall()
    idUni = myresult[0][0]
    #val = (idUni, camposDict.get(row['CAMPOS']))
    val = []
    for k, v in camposDict.items():
        valor = (idUni, v, row[k])
        val.append(valor)
    mycursor.executemany(sqlRank, val)
mydb.commit()